In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

###### 建立一个最简单的网络进行实例

### 1、第一种方法：保存的模型加载后仍能进行修改

In [ ]:
# 模型保存
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
batch_size = 256
n_batch = mnist.train.num_examples // batch_size
# with tf.name_scope('input'):
x = tf.placeholder(tf.float32, [None, 784], name='x-input')
y = tf.placeholder(tf.float32, [None, 10], name='y-input')
# with tf.name_scope('layer'):
# 构建一个784*10的神经网络
w = tf.Variable(tf.truncated_normal([784,10], stddev=0.1))
b = tf.Variable(tf.zeros([10])+0.1)
pred = tf.nn.softmax(tf.matmul(x,w)+b, name='output')
# with tf.name_scope('loss'):
loss = tf.losses.softmax_cross_entropy(y, pred)
train = tf.train.AdamOptimizer(0.01).minimize(loss, name='train')
# with tf.name_scope('accuracy'):
corrent_pred = tf.equal(tf.argmax(y,1), tf.argmax(pred, 1))
accuracy = tf.reduce_mean(tf.cast(corrent_pred, tf.float32), name='accuracy')

saver = tf.train.Saver()
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={x:batch_xs,y:batch_ys})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print('Epoch {}, test accuracy is {}'.format(epoch, acc))
    saver.save(sess, 'models/my_model.ckpt')
    ## 会生成4个文件

In [ ]:
# 模型加载
## ①模型参数和模型文件都有
### 但是这种方法载入模型后需要通过tensor的名字来调用，所以要求模型保存时必须添加name
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
batch_size = 256
n_batch = mnist.train.num_examples // batch_size

with tf.Session() as sess:
    # 载入模型结构
    saver = tf.train.import_meta_graph('models/my_model.ckpt.meta')
    # 载入模型参数
    saver.restore(sess, 'models/my_model.ckpt')
    output = sess.graph.get_tensor_by_name('output:0')    # :0是在保存时自己加上的
    accuracy = sess.graph.get_tensor_by_name('accuracy:0')
    # train是operation，所以需要get_operation_by_name()来调用
    ## 对于模型具体有哪些参数，在载入后怎么调用，那就得看文件发布者是否给出了，没给出就没法调用 ## 
#     for var in tf.trainable_variables():
#         print(var.name)
    train_step = sess.graph.get_operation_by_name('train')
    print(sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels}))
    
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print('Epoch {}, test accuracy is {}'.format(epoch, acc))

In [1]:
# ②模型加载的第二种方法
## 没有模型结构文件但知道模型结构时，可以使用
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
batch_size = 256
n_batch = mnist.train.num_examples // batch_size

# 模型参数必须和原来的参数一样
x = tf.placeholder(tf.float32, [None,784])
y = tf.placeholder(tf.float32, [None,10])
w = tf.Variable(tf.truncated_normal([784,10], stddev=0.1))
b = tf.Variable(tf.zeros([10])+0.1)
pred = tf.nn.softmax(tf.matmul(x,w)+b)
loss = tf.losses.softmax_cross_entropy(y, pred)
train = tf.train.AdamOptimizer(0.001).minimize(loss)
correct_pred = tf.equal(tf.argmax(y,1), tf.argmax(pred,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
saver = tf.train.Saver(max_to_keep=5)    # 只保存最新的5个模型，旧的模型会被覆盖
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(accuracy, feed_dict={x:mnist.test.images,y:mnist.test.labels}))
    saver.restore(sess, 'models/my_model.ckpt')
    print(sess.run(accuracy, feed_dict={x:mnist.test.images,y:mnist.test.labels}))
    for epoch in range(11):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={x:batch_xs, y:batch_ys})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print('Epoch {}, test accuracy is {}'.format(epoch, acc))
        saver.save(sess, 'models/my_model.ckpt', global_step=epoch)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
0.1011
INFO:tensorflow:Restoring parameters from models/my_model.ckpt
0.9338
Epoch 0, test accuracy is 0.9330000281333923
Epoch 1, test accuracy is 0.9332000017166138
Epoch 2, test accuracy is 0.9333999752998352
Epoch 3, test accuracy is 0.9337000250816345
Epoch 4, test accuracy is 0.9332000017166138

### 2、模型保存和加载的第二种方法
（这种只能用，不能再进行训练）

In [ ]:
# 模型保存
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
batch_size = 64
n_batch = mnist.train.num_examples // batch_size

x = tf.placeholder(tf.float32,[None,784], name='x-input')
y = tf.placeholder(tf.float32,[None,10], name='y-input')

W = tf.Variable(tf.truncated_normal([784,10],stddev=0.1))
b = tf.Variable(tf.zeros([10])+0.1)
prediction = tf.nn.softmax(tf.matmul(x,W)+b, name='output')

loss = tf.losses.softmax_cross_entropy(y,prediction)
train_step = tf.train.AdamOptimizer(0.001).minimize(loss, name='train')

correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32), name='accuracy')

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(11):
        for batch in range(n_batch):
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Epoch " + str(epoch) + ",Testing Accuracy " + str(acc))
    ## 保存模型参数和结构,把变量变成常量
    ## output_node_names设置可以输出的tensor
    output_graph_def = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, output_node_names=['output','accuracy'])
    # 保存模型到目录下的models文件夹中
    with tf.gfile.FastGFile('pb_models/my_model.pb',mode='wb') as f:
        f.write(output_graph_def.SerializeToString())

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Epoch 0,Testing Accuracy 0.9029
Epoch 1,Testing Accuracy 0.9138
Epoch 2,Testing Accuracy 0.9199
Epoch 3,Testing Accuracy 0.9231
Epoch 4,Testing Accuracy 0.924
Epoch 5,Testing Accuracy 0.9258
Epoch 6,Testing Accuracy 0.9279
Epoch 7,Testing Accuracy 0.9279
Epoch 8,Testing Accuracy 0.9286
Epoch 9,Testing Accuracy 0.9287
Epoch 10,Testing Accuracy 0.9292
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.


In [ ]:
# 模型加载
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# 载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
# 载入模型
with tf.gfile.FastGFile('pb_models/my_model.pb', 'rb') as f:
    # 创建一个图
    graph_def = tf.GraphDef()
    # 把模型文件载入到图中
    graph_def.ParseFromString(f.read())
    # 载入图到当前环境中
    tf.import_graph_def(graph_def, name='')

with tf.Session() as sess:
    # 根据tensor的名字获取到对应的tensor
    # 之前保存模型的时候模型输出保存为output，":0"是保存模型参数时自动加上的，所以这里也要写上
    output = sess.graph.get_tensor_by_name('output:0')
    # 根据tensor的名字获取到对应的tensor
    # 之前保存模型的时候准确率计算保存为accuracy，":0"是保存模型参数时自动加上的，所以这里也要写上
    accuracy = sess.graph.get_tensor_by_name('accuracy:0')
    # 预测准确率
    print(sess.run(accuracy,feed_dict={'x-input:0':mnist.test.images,'y-input:0':mnist.test.labels}))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


本文亮点：  
模型的保存和加载：  
1、保存的模型加载出来后可进行修改：  
①模型保存：  

    # 其他类似
    saver = tf.train.Saver()
    with tf.Session() as sess:
        ## 其他类似
        saver.save('路径.ckpt')
        ## 会生成4个文件

②模型加载